In [1]:
%load_ext autoreload
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from dbmaster import MasterParams, MasterPersist
from dbbase import MusicDBIDModVal, MusicDBDir, MusicDBData
from dbnote import DownloadRecord, MergeSearchArtist, backup
from utils import FileIO, DirInfo, FileInfo, getFlatList, Timestat, TermTime, TermTimeTS, getTT
from pandas import Series, DataFrame, concat, Timestamp
from pandb import PanDBIO
from musicdb.deezer import MusicDBParams, RawAPIData, MusicDBIO
from os import getpid
from functools import partial

mv = MusicDBIDModVal()
io = FileIO()
mpar = MusicDBParams()
dbio = MusicDBIO()
apiio = RawAPIData()
db = mpar.db

In [3]:
searchArtistRecord = DownloadRecord(db=db, name="SearchArtist", rTypes=["Index", "Data"])
downloadArtistRelatedRecord = DownloadRecord(db=db, name="DownloadArtistRelated", rTypes=["Index", "Data"])
downloadAlbumRecord = DownloadRecord(db=db, name="DownloadAlbum", rTypes=["Index"])
knownArtists = dbio.rdio.getSummaryNameData
allArtists = partial(dbio.rdio.getData, "SearchArtist")

DownloadRecord(db=Deezer, name=SearchArtist, rTypes=['Index', 'Data'])
DownloadRecord(db=Deezer, name=DownloadArtistRelated, rTypes=['Index', 'Data'])
DownloadRecord(db=Deezer, name=DownloadAlbum, rTypes=['Index'])


In [ ]:
if False:
    from utils import FileInfo
    import shutil
    db = dbio.params.db
    dbdir = dbio.params.dir
    nameMap = {"MasterArtists": "SearchArtist", "LocalArtists": "DownloadArtist", "LocalAlbums": "DownloadAlbum"}
    nameMap = nameMap | {"LocalRelated": "DownloadArtistRelated", "LocalArtistsInfo": "DownloadArtistInfo", }
    dinfo = DirInfo(f"/Users/tgadfort/anaconda3/envs/post/pandb/musicdb/{db}")
    
    cpMap = {}
    for origname, newname in nameMap.items():
        cpMap[f"{dbdir}SearchedFor{origname}"] = f"{db}-{newname}-Index"
        cpMap[f"{dbdir}SearchedFor{origname}Data"] = f"{db}-{newname}-Data"
        cpMap[f"{dbdir}SearchedFor{origname}Errors"] = f"{db}-{newname}-Error"
    assert dinfo.exists(), f"{dinfo} does not exist!"
    for srcFile, dstFile in cpMap.items():
        srcFile = dinfo.join(f"{srcFile}.p")
        dstFile = dinfo.join(f"{dstFile}.p")
        if srcFile.exists():
            srcFile.mvFile(dstFile)

In [4]:
##########################################################################################
# Show Summary
##########################################################################################
print(f"{db} Search Results (PID={getpid()})".format(db))
searchArtistRecord.info()
downloadArtistRelatedRecord.info()
downloadAlbumRecord.info()
print(f"  {'KnownArtist Data': <20}: {knownArtists().shape[0]}")

Deezer Search Results (PID=85931)
  SearchArtist Index  : 1086163
  SearchArtist Data   : 0
  SearchArtist Error  : 143
  DownloadArtistRelated Index: 1027060
  DownloadArtistRelated Data: 22150
  DownloadArtistRelated Error: 0
  DownloadAlbum Index : 3716260
  DownloadAlbum Error : 0
  KnownArtist Data    : 1078964


# Search For New Artists

In [5]:
######################################################################################################
# Explode Recent Data
######################################################################################################
def getNewData(searchArtistRecord):    
    newData = searchArtistRecord.getData()
    if len(newData) == 0:
        return DataFrame()
    df = concat([DataFrame(s).T for s in newData.values()])
    df.index = df.index.astype(str)
    newData = df[~df.index.duplicated()].drop("id", axis=1)
    return newData

def mergeNewData(searchArtistRecord):
    searchArtistRecord.load(verbose=False)
    newData = getNewData(searchArtistRecord)
    msr = MergeSearchArtist(db)
    msr.mergeLocal(searchArtistRecord, newData, test=False)

In [10]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = PanDBIO().getUniqueUnmatchedArtistNames(db)
mergeNewData(searchArtistRecord)
searchedNames = Series(searchArtistRecord.getIndex())
artistNamesToGet = knownNames[~knownNames.map(searchArtistRecord.isKnown)]

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {searchedNames.shape[0]}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del searchedNames
del knownNames

/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
****************************************************************************************************************************
*                                               MergeSearchArtist(db=Deezer)                                               *
****************************************************************************************************************************
  ==> Creating copy of local SearchArtist data
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


  ==> Created copy of local SearchArtist data with (2893642, 8) shape
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Found 8850 Searches
Found 29215 Results
Found 0 Duplicate Results
Found 2893642 Previously Known Results
Found 15446 New Results
  ==> 3.30 All Results / Search
  ==> 1.75 New Results / Search
Saving 2909088 Data ... Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]


/Users/tgadfort/code/dbmaster/src/dbmaster/paths.py:22: UserWarning: Perm Path [DirInfo(path=/Volumes/Piggy)] Does Not Exist.
  warnings.warn(f"Perm Path [{permDrivePath}] Does Not Exist.")


Done
Setting Raw Path To [DirInfo(path=/Volumes/Seagate)]
*****************************************************************************************************************************
*                                                 Initializing SearchArtist                                                 *
*****************************************************************************************************************************
  Initializing Data   ... Done
# Deezer Search Results (PID=65628)
#   Available Names:     1244766
#   Known Artist Names:  1086163
#   Artist Names To Get: 197934


In [9]:
ts = Timestat(f"Getting {db} ArtistIDs")
tt = getTT(skipEOD=False, vacation=True)
assert dbio.rdio.isLocal, f"MusicDBIO is not set for local downloads!"

def isError(artistName, nErrors, sleeptime, error=None):
    searchArtistRecord.setError(index=artistName)
    print(f"Search Error ==> {artistName}: {error}")
    nErrors.append(artistName)
    apiio.sleep(sleeptime)
    
n = 0
maxN = 2500000
nErrors = []
searchArtistRecord.load(verbose=False)
for i, (idx, artistName) in enumerate(artistNamesToGet.items()):
    if searchArtistRecord.isKnown(artistName):
        continue

    if len(nErrors) >= 5:
        print("Stopping due to 5 consecutive errors")
        break

    try:
        response = apiio.getArtistSearchData(artistName=artistName)
    except Exception as error:
        isError(artistName, nErrors, 5, error)
        continue

    if not isinstance(response, dict):
        isError(artistName, nErrors, 2.5, "NotDict")
        continue

    nErrors = []
    searchArtistRecord.setData(index=artistName, data=response)
    apiio.sleep(2.5)
    n += 1
        
    if n % 25 == 0:
        ts.update(n=n)
        searchArtistRecord.save()
        apiio.wait(5.0)
        if tt.isFinished():
            break
    
    if n >= maxN:
        print(f"Breaking after {max} downloads...")
        break

ts.stop()
searchArtistRecord.save()

 [Getting Deezer ArtistIDs] | StartTime 2023-11-24 10:59:48
   ====> Terminate Time Set To 2023-11-24 19:00:00 <====
   ====> Will Terminate Process 8 Hour and 11 Second From Now
Searching For Lenny Wolpe                                       7
Searching For Max'1 & the Rootsmaker                            1
Searching For Joshuah De Jesus & Buena Vibra Sextet             1
Searching For Caroline Vasicek                                  3
Searching For Nancy Kleurenblind & De Zingende Roadie           1
Searching For Peter Kliment                                     1
Searching For Arie Pradina                                      1
Searching For Reverss                                           1
Searching For Marco Lovei                                       3
Searching For Christine Meyer                                   3
Searching For John Blitheman                                    0
Searching For Aivars Brīze                                     0
Searching For Jdubs          

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For The Valla                                         6
Searching For Yankey Boy                                        2
Searching For Mayer Malik                                       1
Searching For Bad Beginnings                                    1
Searching For Dayana Orozco                                     1
Searching For Silvernite                                        1
Searching For Kst kshitij                                       1
Searching For Barney Durham                                     1
Searching For Relly Tha Prophet                                 1
Searching For Francuz Mordo                                     2
Searching For Scientist Bitch                                   1
Searching For I KOKO                                            2
Searching For Jomindo                                           1
Searching For DoctorSiren                                       1
Searching For Cassette Tape                                     13
Searching

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For Pino Pacino                                       0
Searching For AirMiah                                           1
Searching For Bilal Hancı                                       1
Searching For Dudes and Their Guys                              1
Searching For Maya Hirasedo                                     1
Searching For Beggo                                             6
Searching For Eric Badanti                                      1
Searching For Super Natural Selection                           1
Searching For Ex Habit                                          1
Searching For Briston Maroney But Different                     1
Searching For Firstborn Unicorn                                 1
Searching For The Freddies                                      7
Searching For Oda Trøen                                         1
Searching For Béla Németh                                     1
Searching For Hello Lightfoot                                   1
Searching 

...

Searching For Jeremy Word                                       2
Searching For Halfwerk                                          1
Searching For Ulex                                              6
Searching For Veronica Elton                                    1
Searching For Baadwrk                                           4
Searching For Base Research                                     1
Searching For Coco Cool                                         2
Searching For Dmitriy Timofeev                                  1
Searching For Miss Kiyami                                       4
Searching For Dean More                                         1
Searching For Autobooze                                         2
Searching For Scorpio d'Haïti                                  0
Searching For Brandon A. Jacobs                                 1
Searching For Hutu                                              12
Searching For Piers Penfold                                     0
Searching

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

Searching For D.D.King                                          2
Searching For Andry Cristian                                    7
Searching For African Spirit Sa                                 1
Searching For Tropical Santa Fe                                 3
Searching For LIL Vamp                                          8
Searching For AVGVSTVS                                          3
Searching For Stuffy D                                          1
Searching For Tom Dissevelt & Kid Baltan                        0
Searching For DJ Symoz                                          3
Searching For Garbage Frank                                     1
Searching For The Overtonics                                    1
Searching For Ribz                                              4
Searching For Bethany Lamb                                      1
Searching For Mae-I                                             2
Searching For Tommaso Dibello                                   1
Searching 

Waiting:   0%|          | 0/50 [00:00<?, ?it/s]

   ====> Terminate Time [2023-11-24 19:00:00] Is Reached <====
 [Getting Deezer ArtistIDs] | RunTime 8 Hour and 47 Second.  2023-11-24 19:00:36
*****************************************************************************************************************************
*                                                    Saving SearchArtist                                                    *
*****************************************************************************************************************************
  Saving Index [1086163] ... Done
  Saving Data [8850] ... Done
  Saving Error [143] ... Done


In [ ]:
searchArtistRecord.save()

In [11]:
msa = MergeSearchArtist(dbio.db)
msa.copyFromLocal(force=True)

****************************************************************************************************************************
*                                               MergeSearchArtist(db=Deezer)                                               *
****************************************************************************************************************************
New Local Shape:  (2909088, 8)
Old Global Shape: (2619382, 8)
New Global Shape: (2909088, 8)


In [5]:
backup(dbio, test=False, doMod=False, doSearch=True)

 [Backing Up Raw Deezer Data] | StartTime 2023-11-24 20:05:50
 [Backing Up Raw Deezer Data] | ?/?        : RunTime .  >> Checking Raw Path [/Volumes/Piggy/Discog/artists-deezer] <<
 [Backing Up Raw Deezer Data] | ?/?        : RunTime .  >> Checking Mod Path [/Volumes/Seagate/Discog/artists-deezer] <<
Not checking the raw mod path
 [Backing Up Raw Deezer Data] | RunTime 15 Second.  2023-11-24 20:06:05


# Download Related Artists

,name,fans,albums
246791,Drake,21698854,46.0
384236,Ed Sheeran,19268807,127
542,David Guetta,17910780,208.0
384236,Ed Sheeran,17861429,86.0
13,Eminem,15075739,50.0
...,...,...,...
54616592,Musashi Samurai,0,0
74186902,ISE Trell,0,3.0
106869192,Trapgod Musashi,0,5
218133705,MU$A$HI,0,3


In [19]:
dbio = MusicDBIO(verbose=False,local=True,mkDirs=False)
apiio = RawAPIData(debug=False)
knownNames = allArtists()[["name", "fans", "albums"]].sort_values(by="fans", ascending=False)
artistNamesToGet = knownNames[~knownNames.index.map(downloadArtistRelatedRecord.isKnown)]["name"]

print(f"# {db} Search Results (PID={getpid()})")
print(f"#   Available Names:     {knownNames.shape[0]}")
print(f"#   Known Artist Names:  {downloadArtistRelatedRecord.numKnown()}")
print(f"#   Artist Names To Get: {artistNamesToGet.shape[0]}")

del knownNames

# Deezer Search Results (PID=86017)
#   Available Names:     2909088
#   Known Artist Names:  1027060
#   Artist Names To Get: 1806614


# Download Artist Data

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(mio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
df

In [ ]:
artistNames            = searchArtists()
artistNames['IndexModVal'] = artistNames.index.map(mv.getModVal)
localArtistsDict       = localArtists.get()
localArtistsErrorsDict = localArtistsErrors.get()
availableNames         = artistNames[~artistNames.index.isin(localArtistsDict.keys())]
availableNames         = availableNames[~availableNames.index.isin(localArtistsErrorsDict.keys())]
artistNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})


print("# {0} Search Results".format(db))
print("#   Available Names:      {0}".format(len(artistNames)))
print("#   Known Artist Names:   {0}".format(len(localArtistsDict)))
print("#   Artist Names To Get:  {0}".format(len(availableNames)))

del availableNames
del localArtistsErrorsDict
del localArtistsDict
del artistNames

#   Artist Names To Get:  3051

In [ ]:
def saveArtistData(db, localArtistsDict, localArtistsDictErrors):
    print("="*150)
    print(f"Saving {len(localArtistsDict)} {db} Artists Data")
    localArtists.save(data=localArtistsDict)
    print(f"Saving {len(localArtistsDictErrors)} {db} Artists Data Errors")
    localArtistsErrors.save(data=localArtistsDictErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} ArtistIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 2500000
localArtistsDict = localArtists.get()
localArtistsDictErrors = localArtistsErrors.get()
stop = False
nErrors = []
for groupModVal,modValData in artistNamesToGet.iteritems():
    if stop is True:
        break
    for j,(artistID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(artistID) is not None for dct in [localArtistsDict, localArtistsDictErrors]]):
            continue
            
        artistName  = row["Name"]
        artistRef   = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getArtistData(artistName=artistName, artistRef=artistRef)
        except:
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(artistName))
            localArtistsDictErrors[artistID] = True
            nErrors.append(artistID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal  = mv.getModVal(artistID)
        mio.data.saveRawArtistData(data=response, modval=modVal, dbID=artistID)
        localArtistsDict[artistID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveArtistData(db, localArtistsDict, localArtistsDictErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

In [ ]:
if True: saveArtistData(db, localArtistsDict, localArtistsDictErrors)

# Download Album Data

## Create Media Data

In [ ]:
mediaData = {}
for modVal in range(100):
    modValData = mio.data.getModValData(modVal)
    modValMediaData = {}
    for artistID,artistIDData in modValData.iteritems():
        for mediaType,mediaTypeData in artistIDData.media.media.items():
            modValMediaData.update({code: [artistID,media.album,media.url] for code,media in mediaTypeData.items()})
    mediaData.update(modValMediaData)
    if (modVal+1) % 10 == 0:
        print(f"ModVal = {modVal+1}")
        
df = DataFrame(mediaData).T
df.columns = ["ArtistID", "Name", "Ref"]
knownMedia.save(data=df)

## Download Data

In [ ]:
mio   = bandcamp.MusicDBIO(verbose=False,local=True,mkDirs=False)
webio = bandcamp.RawWebData(debug=False)

In [ ]:
useArtist = False
numMaster = 250

knownAlbumsData = knownMedia.get()
knownAlbumsData['IndexModVal'] = knownAlbumsData.index.map(mio.getModVal)
availableNames  = concat([artistIDDF.head(numMaster) for artistID,artistIDDF in knownAlbumsData.groupby(["ArtistID"])])
localAlbumsDict = localAlbums.get()
availableNames  = availableNames[~availableNames.index.isin(localAlbumsDict.keys())]
albumNamesToGet = Series({modVal: modValDF for modVal,modValDF in availableNames.groupby(["IndexModVal"])})

print(f"# {db} Album Search Results")
print(f"#   Available Album IDs:  {knownAlbumsData.shape[0]}")
print(f"#   Known Album IDs:      {len(localAlbumsDict)}")
print(f"#   Albums To Download:   {availableNames.shape[0]}")

del availableNames
del localAlbumsDict
del knownAlbumsData

#   Albums To Download:   23457
#   Albums To Download:   20437
#   Albums To Download:   12137
#   Albums To Download:   4132

In [ ]:
def saveAlbumData(db, localAlbumsDict, searchedForErrors):
    print("="*150)
    print(f"Saving {len(localAlbumsDict)} {db} Albums Data")
    localAlbums.save(data=localAlbumsDict)
    print(f"Saving {len(searchedForErrors)} {db} Searched For Errors")
    errors.save(data=searchedForErrors)
    print("="*150)

In [ ]:
ts = Timestat("Getting {0} AlbumIDs".format(db))
tt = getTT(skipEOD=False)

n    = 0
maxN = 25000000
localAlbumsDict     = localAlbums.get()
searchedForErrors   = errors.get()
stop = False
nErrors = []
for groupModVal,modValData in albumNamesToGet.iteritems():
    if stop is True:
        break
    for j,(albumID,row) in enumerate(modValData.iterrows()):
        if len(nErrors) >= 5:
            for artistID in nErrors:
                print(f"del searchedForErrors['{artistID}']")
                stop=True
                break
        if any([dct.get(albumID) is not None for dct in [localAlbumsDict, searchedForErrors]]):
            continue
            
        artistID   = row["ArtistID"]
        albumName  = row["Name"]
        albumRef  = row["Ref"]

        print(f"{groupModVal: <8} |{j: <8} | {n: <8} | ", end="")
        try:
            response = webio.getAlbumData(albumName=albumName, albumRef=albumRef)
        except:
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(10)
            continue

        if not isinstance(response,bytes):
            print("Error ==> {0}".format(albumName))
            searchedForErrors[albumID] = True
            nErrors.append(albumID)
            webio.sleep(3.5)
            continue

        nErrors = []
        modVal=mio.mv.get(albumID)
        mio.data.saveRawArtistAlbumData(data=response, modval=modVal, dbID=albumID)
        localAlbumsDict[albumID] = True
        webio.sleep(4.5)
        n += 1
        nLastErrors = 0
        
        if n % 5 == 0:
            if tt.isFinished():
                stop=True
                break

        if n % 50 == 0:
            webio.sleep(5)
            
        if n % 100 == 0:
            saveAlbumData(db, localAlbumsDict, searchedForErrors)
            if tt.isFinished():
                stop=True
                break
            webio.wait(10.0)

        if n >= maxN:
            print("Breaking after {0} downloads...".format(maxN))
            stop=True
            break

ts.stop()
if True: saveAlbumData(db, localAlbumsDict, searchedForErrors)

In [ ]:
from os import getpid
getpid()

# Download Lists

## Download Starter

In [ ]:
from apiutils import WebIO
from ioutils import FileIO, HTMLIO
io  = FileIO()
hio = HTMLIO()
wio = WebIO()

In [ ]:
starter             = {}
starter["List"]     = "https://www.bandcamp.org/lists.php"
starter["Genre"]    = "https://www.bandcamp.org/genre.php"
#starterBestAlbum = "https://www.bandcamp.org/ratings/6-highest-rated/2023/1"
starter["Rating"]   = "https://www.bandcamp.org/ratings"
starter["Discover"] = "https://www.bandcamp.org/discover"
starter["Releases"] = "https://www.bandcamp.org/releases"
starter["MustHear"] = "https://www.bandcamp.org/must-hear"

savename = "../../sandbox/AOTYstarter.p"
starterData = io.get(savename)
for key,url in starter.items():
    if starterData.get(key) is not None:
        continue
    retval = wio.get(url)
    if retval.code == 200:
        print(key)
        starterData[key] = retval.data
    wio.sleep(3)
        
print(f"Saving data to {savename}")
io.save(idata=starterData, ifile=savename)

In [ ]:
useStarter = False
useSite    = True

if useStarter is True:
    aotyData = io.get("../../sandbox/AOTYstarter.p")
elif useSite is True:
    aotyData = io.get("../../sandbox/AOTYsiteData.p") | io.get("../../sandbox/AOTYsiteData2.p")
else:
    aotyData = {}
    
N  = len(aotyData)
ts = Timestat(f"Sorting {N} Site Refs")
refsData = {"List": {}, "Lists": {}, "Rating": {}, "Album": {}, "Artist": {}, "Discover": {}, "Genre": {}, "MustHear": {}, "Release": {}, "Spotify": {}, "Apple": {}, "Amazon": {}, "User": {}}
for n,(key,keyData) in enumerate(aotyData.items()):
    if (n+1) % 500 == 0 or (n+1) == 100:
        ts.update(n=n+1, N=N)
        
    bsdata = hio.get(keyData)
    refs = bsdata.findAll("a")
    for ref in refs:
        href = ref.get('href')
        if not isinstance(href,str):
            continue
        if href.startswith("/list/"):
            refsData["List"][href] = ref.text.strip()
        elif "lists.php" in href:
            refsData["Lists"][href] = ref.text.strip()
        elif href.startswith("/ratings/"):
            refsData["Rating"][href] = ref.text.strip()
        elif href.startswith("/artist/"):
            refsData["Artist"][href] = ref.text.strip()
        elif href.startswith("/album/"):
            refsData["Album"][href] = ref.text.strip()
        elif href.startswith("/discover/"):
            refsData["Discover"][href] = ref.text.strip()
        elif href.startswith("/genre/"):
            refsData["Genre"][href] = ref.text.strip()
        elif href.startswith("/must-hear/"):
            refsData["MustHear"][href] = ref.text.strip()
        elif "/releases/" in href:
            refsData["Release"][href] = ref.text.strip()
        elif "spotify.com" in href:
            refsData["Spotify"][href] = ref.text.strip()
        elif "apple.com" in href:
            refsData["Apple"][href] = ref.text.strip()
        elif "amazon.com" in href:
            refsData["Amazon"][href] = ref.text.strip()
        elif "/user/" in href:
            refsData["User"][href] = ref.text.strip()
        else:
            continue
            print(href,'\t|\t',ref.text)
            
ts.stop()

In [ ]:
savename = "../../sandbox/AOTYsiteRefs.p"
print(f"Saving data to {savename}")
for key,keyData in refsData.items():
    print(f"  {key: <20}{len(keyData)}")
io.save(idata=refsData, ifile=savename)

In [ ]:
siteData   = {}
artistData = {}
albumData  = {}
userData   = {}
otherData  = {}

if False:
    for ref,name in refsData["List"].items():
        if ref.startswith("/artist/"):
            artistData[ref] = name
        elif ref.startswith("/album/"):
            albumData[ref] = name
        elif ref.startswith("/user/"):
            userData[ref] = name
        elif ref.startswith("/"):
            siteData[ref] = name
        else:
            otherData[ref] = name
else:
    for key,keyData in refsData.items():
        for ref,name in keyData.items():
            if ref.startswith("/artist/"):
                artistData[ref] = name
            elif ref.startswith("/album/"):
                albumData[ref] = name
            elif ref.startswith("/user/"):
                userData[ref] = name
            elif ref.startswith("/"):
                if "/list/" in ref:
                    siteData[ref] = name
                else:
                    otherData[ref] = name
            else:
                otherData[ref] = name
            
print(f"Found {len(artistData)} Artist Refs")
print(f"Found {len(albumData)} Album Refs")
print(f"Found {len(userData)} User Refs")
print(f"Found {len(siteData)} Site Refs")
print(f"Found {len(otherData)} Other Refs")

In [ ]:
for year in range(1970,2005):
    key = f'/lists.php?y={year}'
    val = 'View More'
    siteData[key] = val

In [ ]:
savename1 = "../../sandbox/AOTYsiteData.p"
siteDataDownloads1 = io.get(savename1)
print(f"Found {len(siteDataDownloads1)} Previous Downloads")
savename2 = "../../sandbox/AOTYsiteData2.p"
siteDataDownloads2 = io.get(savename2)
print(f"Found {len(siteDataDownloads2)} Previous Downloads")
N = len(siteData)
ts = Timestat(f"Downloading {N} Site Refs")
for n,(ref,name) in enumerate(siteData.items()):
    url=f"https://www.bandcamp.org{ref}"
    if any([dct.get(ref) is not None for dct in [siteDataDownloads1,siteDataDownloads2]]):
        continue
    
    retval = wio.get(url)
    if retval.code == 200:
        print(f"{n: <6} | {N: <6} | {ref}")
        siteDataDownloads2[ref] = retval.data
    wio.sleep(3)
    
    if (n+1) % 25 == 0:
        ts.update(n=n+1,N=N)
        print(f"Saving {len(siteDataDownloads2)} data to {savename2}")
        io.save(idata=siteDataDownloads2, ifile=savename2)

ts.stop()
        
print(f"Saving data to {savename2}")
io.save(idata=siteDataDownloads2, ifile=savename2)

In [ ]:
siteDataDownloads = io.get("../../sandbox/AOTYsiteData.p")

In [ ]:
io.save(idata=siteDataDownloads, ifile=savename)

## Check For New Data

In [ ]:
from lib.bandcamp import MusicDBID
mid = MusicDBID()
mid.getAlbumID('/album/515536-beyonce-renaissance/critic-lists/?f=all&y=2022')

In [ ]:
df = DataFrame(Series(refsData["Album"])).reset_index().rename(columns={"index": "Ref", 0: "List"})
df["AlbumID"] = df["Ref"].map(mid.getAlbumID)
df = df[~df["AlbumID"].duplicated()]

In [ ]:
df = DataFrame(Series(refsData["Artist"])).reset_index().rename(columns={"index": "Ref", 0: "Name"})
df["ArtistID"] = df["Ref"].map(mid.getArtistID)
df = df[~df["ArtistID"].duplicated()]

In [ ]:
artistNames = searchArtists()

In [ ]:
df.index = df["ArtistID"]
df = df.drop(["ArtistID"], axis=1)
artistNames = concat([artistNames,df])
artistNames = artistNames[~artistNames.index.duplicated()]

In [ ]:
mio.data.saveSearchArtistData(data=artistNames)

# Backup

In [ ]:
from utils import StoreData, backup
from numpy import array_split
sd = StoreData("bandcamp", "Artist")
for modVals in array_split(range(100), 2):
    sd.mergeLocalData(modVals=modVals)
sd.mergeGlobalData()

In [ ]:
from time import sleep
sleep(200)
sd.mergeGlobalData()

In [ ]:
for modVal in range(67):
    srcDir = DirInfo(f"/Volumes/Piggy/Discog/artists-bandcamp/{modVal}/artists")
    files  = [FileInfo(ifile) for ifile in srcDir.getFiles()]
    files  = [finfo for finfo in files if finfo.basename.isdigit()]
    dstDir = DirInfo(f"/Users/tgadfort/Music/Discog/artists-bandcamp/{modVal}/artists")
    for srcFile in files:
        dstFile = dstDir.join(srcFile.name)
        srcFile.mvFile(dstFile)